In [1]:
import pandas as pd
from glob import glob


# Read single CSV file of acceloremeter data and gyroscope

In [24]:
simple_file_acc = pd.read_csv("../../data/raw/MetaMotion/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Accelerometer_12.500Hz_1.4.4.csv")
simple_file_gyr = pd.read_csv("../../data/raw/MetaMotion/MetaMotion/A-bench-heavy2-rpe8_MetaWear_2019-01-11T16.10.08.270_C42732BE255C_Gyroscope_25.000Hz_1.4.4.csv")
simple_file_gyr

,epoch (ms),time (01:00),elapsed (s),x-axis (deg/s),y-axis (deg/s),z-axis (deg/s)
0,1547219408351,2019-01-11T16:10:08.351,0.00,0.122,-5.488,-3.841
1,1547219408391,2019-01-11T16:10:08.391,0.04,2.195,-9.695,-0.610
2,1547219408431,2019-01-11T16:10:08.431,0.08,2.622,-8.110,-4.024
3,1547219408471,2019-01-11T16:10:08.471,0.12,1.951,-4.695,-4.634
4,1547219408511,2019-01-11T16:10:08.511,0.16,1.524,-2.561,-2.500
...,...,...,...,...,...,...
409,1547219424711,2019-01-11T16:10:24.711,16.36,2.195,-1.402,4.268
410,1547219424751,2019-01-11T16:10:24.751,16.40,4.451,-1.220,0.244
411,1547219424791,2019-01-11T16:10:24.791,16.44,7.195,-4.329,6.524
412,1547219424831,2019-01-11T16:10:24.831,16.48,8.963,-1.037,-0.366


# List all data in data/raw/MetaMotion

In [7]:
files = glob("../../data/raw/MetaMotion/MetaMotion\\*.csv")
len(files)

187

# Extract features from filename (test echantillon)

In [ ]:
path = "../../data/raw/MetaMotion/MetaMotion\\" #on va l'utiliser pour extraire le participant
f = files[0]
participant =f.split("-")[0].replace(path, "")
category = f.split("-")[1]
label = f.split("-")[2].rstrip("123") 

#add columns to the df
df = pd.read_csv(f)
df["participant"] = participant
df["category"] = category
df["label"] = label
df

# Read all files

In [13]:
acc_df = pd.DataFrame()
gyr_df = pd.DataFrame()

acc_set = 1
gyr_set = 1

for f in files:
    
    participant =f.split("-")[0].replace(path, "")
    category = f.split("-")[1]
    label = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

    #add columns to the df
    df = pd.read_csv(f)
    
    df["participant"] = participant
    df["category"] = category
    df["label"] = label
    df["set"] = acc_set
    
    if "Accelerometer" in f:
        df["set"] = acc_set
        acc_set += 1
        acc_df = pd.concat([acc_df, df])
    
    if "Gyroscope" in f:
        df["set"] = gyr_set
        gyr_set += 1
        gyr_df = pd.concat([gyr_df, df])

# Working with datetimes

In [ ]:
acc_df.info()

acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit= "ms")
gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit= "ms")

del acc_df["epoch (ms)"]
del acc_df["time (01:00)"]
del acc_df["elapsed (s)"]

del gyr_df["epoch (ms)"]
del gyr_df["time (01:00)"]
del gyr_df["elapsed (s)"]

# Turn into function

In [14]:
files = glob("../../data/raw/MetaMotion/MetaMotion\\*.csv")
def read_data_from_files(files):
    
    acc_df = pd.DataFrame()
    gyr_df = pd.DataFrame()

    acc_set = 1
    gyr_set = 1

    for f in files:
        
        participant =f.split("-")[0].replace(path, "")
        category = f.split("-")[1]
        label = f.split("-")[2].rstrip("123").rstrip("_MetaWear_2019")

        #add columns to the df
        df = pd.read_csv(f)
        
        df["participant"] = participant
        df["category"] = category
        df["label"] = label
        
        if "Accelerometer" in f:
            df["set"] = acc_set
            acc_set += 1
            acc_df = pd.concat([acc_df, df])
        
        if "Gyroscope" in f:
            df["set"] = gyr_set
            gyr_set += 1
            gyr_df = pd.concat([gyr_df, df])
        
    acc_df.index = pd.to_datetime(acc_df["epoch (ms)"], unit= "ms")
    gyr_df.index = pd.to_datetime(gyr_df["epoch (ms)"], unit= "ms")

    del acc_df["epoch (ms)"]
    del acc_df["time (01:00)"]
    del acc_df["elapsed (s)"]

    del gyr_df["epoch (ms)"]
    del gyr_df["time (01:00)"]
    del gyr_df["elapsed (s)"]
    
    return acc_df, gyr_df
    
acc_df , gyr_df = read_data_from_files(files)

# Merging datasets

In [15]:
data_merged = pd.concat([acc_df.iloc[:,:3], gyr_df], axis=1)

#rename columns 
data_merged.columns = [
    "acc_x",
    "acc_y",
    "acc_z",
    "gyr_x",
    "gyr_y",
    "gyr_z",
    "participant",
    "label",
    "category",
    "set",
    ]

In [16]:
data_merged

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:04.950,NaN,NaN,NaN,-10.671,-1.524,5.976,B,bench,heavy,30.0
2019-01-11 15:08:04.990,NaN,NaN,NaN,-8.720,-2.073,3.171,B,bench,heavy,30.0
2019-01-11 15:08:05.030,NaN,NaN,NaN,0.488,-3.537,-4.146,B,bench,heavy,30.0
2019-01-11 15:08:05.070,NaN,NaN,NaN,0.244,-5.854,3.537,B,bench,heavy,30.0
2019-01-11 15:08:05.110,NaN,NaN,NaN,-0.915,0.061,-2.805,B,bench,heavy,30.0
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:35:13.382,-0.060,-1.021,-0.058,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-20 17:35:13.462,-0.035,-1.037,-0.026,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2019-01-20 17:35:13.542,-0.045,-1.029,-0.033,NaN,NaN,NaN,NaN,NaN,NaN,NaN


# Resample data (frequency conversion)

In [17]:
sampling = {
    "acc_x": "mean",
    "acc_y": "mean",
    "acc_z": "mean",
    "gyr_x": "mean",
    "gyr_y": "mean",
    "gyr_z": "mean",
    "participant": "last",
    "label": "last",
    "category": "last",
    "set": "last",
    
}

data_merged[:1000].resample(rule="200ms").apply(sampling)

days = [g for n, g in data_merged.groupby(pd.Grouper(freq="D"))]
data_resampled = pd.concat([df.resample(rule="200ms").apply(sampling).dropna() for df in days])
data_resampled.info()
data_resampled["set"] = data_resampled["set"].astype("int")

<class 'pandas.core.frame.DataFrame'>
DatetimeIndex: 9009 entries, 2019-01-11 15:08:05.200000 to 2019-01-20 17:33:27.800000
Data columns (total 10 columns):
 #   Column       Non-Null Count  Dtype  
---  ------       --------------  -----  
 0   acc_x        9009 non-null   float64
 1   acc_y        9009 non-null   float64
 2   acc_z        9009 non-null   float64
 3   gyr_x        9009 non-null   float64
 4   gyr_y        9009 non-null   float64
 5   gyr_z        9009 non-null   float64
 6   participant  9009 non-null   object 
 7   label        9009 non-null   object 
 8   category     9009 non-null   object 
 9   set          9009 non-null   float64
dtypes: float64(7), object(3)
memory usage: 774.2+ KB


In [ ]:
# Accelerometer:    12.500HZ
# Gyroscope:        25.000Hz
data_resampled

,acc_x,acc_y,acc_z,gyr_x,gyr_y,gyr_z,participant,label,category,set
epoch (ms),,,,,,,,,,
2019-01-11 15:08:05.200,0.013500,0.977000,-0.071000,-1.8904,2.4392,0.9388,B,bench,heavy,30
2019-01-11 15:08:05.400,-0.001500,0.970500,-0.079500,-1.6826,-0.8904,2.1708,B,bench,heavy,30
2019-01-11 15:08:05.600,0.001333,0.971667,-0.064333,2.5608,-0.2560,-1.4146,B,bench,heavy,30
2019-01-11 15:08:05.800,-0.024000,0.957000,-0.073500,8.0610,-4.5244,-2.0730,B,bench,heavy,30
2019-01-11 15:08:06.000,-0.028000,0.957667,-0.115000,2.4390,-1.5486,-3.6098,B,bench,heavy,30
...,...,...,...,...,...,...,...,...,...,...
2019-01-20 17:33:27.000,-0.048000,-1.041500,-0.076500,1.4146,-5.6218,0.2926,E,row,medium,90
2019-01-20 17:33:27.200,-0.037000,-1.030333,-0.053333,-2.7684,-0.5854,2.2440,E,row,medium,90
2019-01-20 17:33:27.400,-0.060000,-1.031000,-0.082000,2.8416,-5.1342,-0.1220,E,row,medium,90


# Export dataset

In [19]:
data_resampled.to_pickle("../../data/interim/O1_data_processed.pkl")